<h1 align="center">Da Best <br>Spotify Recommendation System</h1>

In [1]:
import pandas as pd
import numpy as np
import re

from sklearn.feature_extraction.text import TfidfVectorizer

<h2 align="left">Data Pre-processing</h2>


In [2]:
data_main = pd.read_csv('Datasets/data.csv')
data_main.head(0)

,valence,year,acousticness,artists,danceability,duration_ms,energy,explicit,id,instrumentalness,key,liveness,loudness,mode,name,popularity,release_date,speechiness,tempo


In [3]:
genre_data = pd.read_csv('Datasets/data_w_genres.csv')
genre_data.head(0)

,genres,artists,acousticness,danceability,duration_ms,energy,instrumentalness,liveness,loudness,speechiness,tempo,valence,popularity,key,mode,count


In [4]:
genre_data['genres'].values

array(["['show tunes']", '[]', '[]', ..., '[]',
       "['chinese indie', 'chinese indie rock']",
       "['classic korean pop']"], dtype=object)

regex statement to extract the genre and input into a list

In [5]:
genre_data['genres_upd'] = genre_data['genres'].apply(lambda x: [re.sub(' ','_',i) for i in re.findall(r"'([^']*)'", x)])
genre_data.genres_upd.values

array([list(['show_tunes']), list([]), list([]), ..., list([]),
       list(['chinese_indie', 'chinese_indie_rock']),
       list(['classic_korean_pop'])], dtype=object)

Similarly converting the array of string to an array of lists to the main data

In [6]:

data_main['artists_upd1'] = data_main['artists'].apply(lambda x: re.findall(r"'([^']*)'", x))
data_main['artists_upd1'].values


array([list(['Sergei Rachmaninoff', 'James Levine', 'Berliner Philharmoniker']),
       list(['Dennis Day']),
       list(['KHP Kridhamardawa Karaton Ngayogyakarta Hadiningrat']), ...,
       list(['MAMAMOO']), list(['Eminem']), list(['KEVVO', 'J Balvin'])],
      dtype=object)

In [7]:
data_main['artists_upd2'] = data_main['artists'].apply(lambda x: re.findall('\"(.*?)\"',x))
data_main['artists_upd'] = np.where(data_main['artists_upd1'].apply(lambda x: not x), data_main['artists_upd2'], data_main['artists_upd1'] )

In [8]:
data_main.head()

,valence,year,acousticness,artists,danceability,duration_ms,energy,explicit,id,instrumentalness,...,loudness,mode,name,popularity,release_date,speechiness,tempo,artists_upd1,artists_upd2,artists_upd
0,0.0594,1921,0.982,"['Sergei Rachmaninoff', 'James Levine', 'Berli...",0.279,831667,0.211,0,4BJqT0PrAfrxzMOxytFOIz,0.878000,...,-20.096,1,"Piano Concerto No. 3 in D Minor, Op. 30: III. ...",4,1921,0.0366,80.954,"[Sergei Rachmaninoff, James Levine, Berliner P...",[],"[Sergei Rachmaninoff, James Levine, Berliner P..."
1,0.9630,1921,0.732,['Dennis Day'],0.819,180533,0.341,0,7xPhfUan2yNtyFG0cUWkt8,0.000000,...,-12.441,1,Clancy Lowered the Boom,5,1921,0.4150,60.936,[Dennis Day],[],[Dennis Day]
2,0.0394,1921,0.961,['KHP Kridhamardawa Karaton Ngayogyakarta Hadi...,0.328,500062,0.166,0,1o6I8BglA6ylDMrIELygv1,0.913000,...,-14.850,1,Gati Bali,5,1921,0.0339,110.339,[KHP Kridhamardawa Karaton Ngayogyakarta Hadin...,[],[KHP Kridhamardawa Karaton Ngayogyakarta Hadin...
3,0.1650,1921,0.967,['Frank Parker'],0.275,210000,0.309,0,3ftBPsC5vPBKxYSee08FDH,0.000028,...,-9.316,1,Danny Boy,3,1921,0.0354,100.109,[Frank Parker],[],[Frank Parker]
4,0.2530,1921,0.957,['Phil Regan'],0.418,166693,0.193,0,4d6HGyGT8e121BsdKmw9v6,0.000002,...,-10.096,1,When Irish Eyes Are Smiling,2,1921,0.0380,101.665,[Phil Regan],[],[Phil Regan]


In [9]:
data_main['artists_song'] = data_main.apply(lambda row: row['artists_upd'][0]+row['name'],axis = 1)
data_main['artists_song'][0]
data_main.sort_values(['artists_song','release_date'], ascending = False, inplace = True)

In [10]:
data_main[data_main['name']=='Shape of You']

,valence,year,acousticness,artists,danceability,duration_ms,energy,explicit,id,instrumentalness,...,mode,name,popularity,release_date,speechiness,tempo,artists_upd1,artists_upd2,artists_upd,artists_song
19074,0.931,2017,0.581,['Ed Sheeran'],0.825,233713,0.652,0,7qiZfU4dY1lWllzX7mPBI3,0.0,...,0,Shape of You,85,2017-03-03,0.0802,95.977,[Ed Sheeran],[],[Ed Sheeran],Ed SheeranShape of You
74616,0.931,2017,0.581,['Ed Sheeran'],0.825,233713,0.652,0,0FE9t6xYkqWXU2ahLh6D8X,0.0,...,0,Shape of You,73,2017-01-06,0.0802,95.977,[Ed Sheeran],[],[Ed Sheeran],Ed SheeranShape of You


In [11]:
data_main.drop_duplicates('artists_song',inplace = True)

In [12]:
data_main[data_main['name']=='Shape of You']

,valence,year,acousticness,artists,danceability,duration_ms,energy,explicit,id,instrumentalness,...,mode,name,popularity,release_date,speechiness,tempo,artists_upd1,artists_upd2,artists_upd,artists_song
19074,0.931,2017,0.581,['Ed Sheeran'],0.825,233713,0.652,0,7qiZfU4dY1lWllzX7mPBI3,0.0,...,0,Shape of You,85,2017-03-03,0.0802,95.977,[Ed Sheeran],[],[Ed Sheeran],Ed SheeranShape of You
